# Дополнительное задание по проекту 04 "Авиарейсы без потерь".

## Задача: принять решение о том, какие рейсы зимнего периода стоит исключить из расписания.

## Цели: 

- Указать номера рейсов, которые можно убрать. Требуется также указать предполагаемую прибыльность этих рейсов и среднюю прибыльность всех рейсов.

- Описать, как проводилась аналитика, какие данные были использованы для оценки доходности рейса и расходов на них.

Импортируем нужные библиотеки.

In [1]:
import pandas as pd
pd.options.display.max_columns = 100

Считываем набор данных.

In [2]:
df = pd.read_csv('sf_data.csv')

Посмотрим на него.

In [3]:
df.head(10)

,aircraft_code,flight_id,flight_no,scheduled_departure,scheduled_arrival,departure_airport,arrival_airport,status,actual_departure,actual_arrival,seats_count,business_count,economy_count,ticket_economy,ticket_comfort,ticket_bisiness,ticket_total,economy_amount,comfort_amount,business_amount,total_amount,fuel_consumption_per_hour,fullness,flight_time_hours,fuel_price_per_ton
0,733,136119,PG0252,2017-01-03T10:05:00Z,2017-01-03T11:45:00Z,AAQ,SVO,Arrived,2017-01-03T10:08:00Z,2017-01-03T11:49:00Z,130,12,118,102.0,0.0,11.0,113.0,1250400.0,NaN,402600.0,1653000.0,2.6,0.869231,1.666667,51300
1,733,136120,PG0252,2017-02-24T10:05:00Z,2017-02-24T11:45:00Z,AAQ,SVO,Arrived,2017-02-24T10:07:00Z,2017-02-24T11:46:00Z,130,12,118,98.0,0.0,11.0,109.0,1202800.0,NaN,402600.0,1605400.0,2.6,0.838462,1.666667,51300
2,733,136122,PG0252,2017-01-08T10:05:00Z,2017-01-08T11:45:00Z,AAQ,SVO,Arrived,2017-01-08T10:08:00Z,2017-01-08T11:48:00Z,130,12,118,87.0,0.0,10.0,97.0,1065000.0,NaN,366000.0,1431000.0,2.6,0.746154,1.666667,51300
3,733,136130,PG0252,2017-02-13T10:05:00Z,2017-02-13T11:45:00Z,AAQ,SVO,Arrived,2017-02-13T10:07:00Z,2017-02-13T11:46:00Z,130,12,118,97.0,0.0,10.0,107.0,1190600.0,NaN,366000.0,1556600.0,2.6,0.823077,1.666667,51300
4,733,136131,PG0252,2017-01-28T10:05:00Z,2017-01-28T11:45:00Z,AAQ,SVO,Arrived,2017-01-28T10:07:00Z,2017-01-28T11:46:00Z,130,12,118,112.0,0.0,12.0,124.0,1373600.0,NaN,439200.0,1812800.0,2.6,0.953846,1.666667,51300
5,733,136132,PG0252,2017-02-15T10:05:00Z,2017-02-15T11:45:00Z,AAQ,SVO,Arrived,2017-02-15T10:09:00Z,2017-02-15T11:48:00Z,130,12,118,104.0,0.0,12.0,116.0,1274800.0,NaN,439200.0,1714000.0,2.6,0.892308,1.666667,51300
6,733,136135,PG0252,2017-02-01T10:05:00Z,2017-02-01T11:45:00Z,AAQ,SVO,Arrived,2017-02-01T10:08:00Z,2017-02-01T11:48:00Z,130,12,118,104.0,0.0,11.0,115.0,1276000.0,NaN,402600.0,1678600.0,2.6,0.884615,1.666667,51300
7,733,136146,PG0252,2017-02-09T10:05:00Z,2017-02-09T11:45:00Z,AAQ,SVO,Arrived,2017-02-09T10:08:00Z,2017-02-09T11:48:00Z,130,12,118,98.0,0.0,11.0,109.0,1201600.0,NaN,402600.0,1604200.0,2.6,0.838462,1.666667,51300
8,733,136159,PG0252,2017-01-04T10:05:00Z,2017-01-04T11:45:00Z,AAQ,SVO,Arrived,2017-01-04T10:07:00Z,2017-01-04T11:45:00Z,130,12,118,103.0,0.0,12.0,115.0,1261400.0,NaN,439200.0,1700600.0,2.6,0.884615,1.666667,51300
9,733,136164,PG0252,2017-01-07T10:05:00Z,2017-01-07T11:45:00Z,AAQ,SVO,Arrived,2017-01-07T10:07:00Z,2017-01-07T11:46:00Z,130,12,118,109.0,0.0,11.0,120.0,1335800.0,NaN,402600.0,1738400.0,2.6,0.923077,1.666667,51300


Изучим общую информацию о наборе данных.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   aircraft_code              127 non-null    object 
 1   flight_id                  127 non-null    int64  
 2   flight_no                  127 non-null    object 
 3   scheduled_departure        127 non-null    object 
 4   scheduled_arrival          127 non-null    object 
 5   departure_airport          127 non-null    object 
 6   arrival_airport            127 non-null    object 
 7   status                     127 non-null    object 
 8   actual_departure           127 non-null    object 
 9   actual_arrival             127 non-null    object 
 10  seats_count                127 non-null    int64  
 11  business_count             127 non-null    int64  
 12  economy_count              127 non-null    int64  
 13  ticket_economy             118 non-null    float64

В наборе данных представлена информация о 127 рейсах.

Видим, что поле comfort_amount пустое - это логично, так как в самолётах с кодами 733 и SU9 представлены места только эконом- и бизнес-классов. 

Также отметим пропущенные значения в полях, отвечающих за количество проданных билетов и значение выручки от продажи билетов. 

Посмотрим на них подробнее.

In [5]:
df[df['total_amount'].isna()]

,aircraft_code,flight_id,flight_no,scheduled_departure,scheduled_arrival,departure_airport,arrival_airport,status,actual_departure,actual_arrival,seats_count,business_count,economy_count,ticket_economy,ticket_comfort,ticket_bisiness,ticket_total,economy_amount,comfort_amount,business_amount,total_amount,fuel_consumption_per_hour,fullness,flight_time_hours,fuel_price_per_ton
118,733,136523,PG0194,2017-01-10T06:10:00Z,2017-01-10T11:15:00Z,AAQ,NOZ,Arrived,2017-01-10T06:14:00Z,2017-01-10T11:26:00Z,130,12,118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.6,NaN,5.083333,51300
119,733,136513,PG0194,2017-02-28T06:10:00Z,2017-02-28T11:15:00Z,AAQ,NOZ,Arrived,2017-02-28T06:13:00Z,2017-02-28T11:23:00Z,130,12,118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.6,NaN,5.083333,51300
120,733,136514,PG0194,2017-02-07T06:10:00Z,2017-02-07T11:15:00Z,AAQ,NOZ,Arrived,2017-02-07T06:14:00Z,2017-02-07T11:13:00Z,130,12,118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.6,NaN,5.083333,51300
121,733,136560,PG0194,2017-01-24T06:10:00Z,2017-01-24T11:15:00Z,AAQ,NOZ,Arrived,2017-01-24T06:13:00Z,2017-01-24T11:18:00Z,130,12,118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.6,NaN,5.083333,51300
122,733,136544,PG0194,2017-01-31T06:10:00Z,2017-01-31T11:15:00Z,AAQ,NOZ,Arrived,2017-01-31T06:10:00Z,2017-01-31T11:14:00Z,130,12,118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.6,NaN,5.083333,51300
123,733,136511,PG0194,2017-02-21T06:10:00Z,2017-02-21T11:15:00Z,AAQ,NOZ,Arrived,2017-02-21T06:15:00Z,2017-02-21T11:25:00Z,130,12,118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.6,NaN,5.083333,51300
124,733,136540,PG0194,2017-01-17T06:10:00Z,2017-01-17T11:15:00Z,AAQ,NOZ,Arrived,2017-01-17T06:14:00Z,2017-01-17T11:21:00Z,130,12,118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.6,NaN,5.083333,51300
125,733,136546,PG0194,2017-01-03T06:10:00Z,2017-01-03T11:15:00Z,AAQ,NOZ,Arrived,2017-01-03T06:13:00Z,2017-01-03T11:15:00Z,130,12,118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.6,NaN,5.083333,51300
126,733,136567,PG0194,2017-02-14T06:10:00Z,2017-02-14T11:15:00Z,AAQ,NOZ,Arrived,2017-02-14T06:12:00Z,2017-02-14T11:23:00Z,130,12,118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.6,NaN,5.083333,51300


Видим 9 рейсов, по которым отсутствуют количество проданных билетов и значение выручки от продажи билетов. Приходим к выводу об отсутствии данных по рейсу Анапа-Новокузнецк. 

В числе возможных причин пропущенных значений в этих полях:

- данные не были загружены в базу данных;
- рейс совершается, несмотря на отсутствие пассажиров на рейсе.

Нам интересны состоявшиеся рейсы, которые генерируют выручку. Поэтому рассмотрим рейсы, кроме рейса Анапа-Новокузнецк.

In [6]:
df = df[df['arrival_airport'] != 'NOZ']
df.head(10)

,aircraft_code,flight_id,flight_no,scheduled_departure,scheduled_arrival,departure_airport,arrival_airport,status,actual_departure,actual_arrival,seats_count,business_count,economy_count,ticket_economy,ticket_comfort,ticket_bisiness,ticket_total,economy_amount,comfort_amount,business_amount,total_amount,fuel_consumption_per_hour,fullness,flight_time_hours,fuel_price_per_ton
0,733,136119,PG0252,2017-01-03T10:05:00Z,2017-01-03T11:45:00Z,AAQ,SVO,Arrived,2017-01-03T10:08:00Z,2017-01-03T11:49:00Z,130,12,118,102.0,0.0,11.0,113.0,1250400.0,NaN,402600.0,1653000.0,2.6,0.869231,1.666667,51300
1,733,136120,PG0252,2017-02-24T10:05:00Z,2017-02-24T11:45:00Z,AAQ,SVO,Arrived,2017-02-24T10:07:00Z,2017-02-24T11:46:00Z,130,12,118,98.0,0.0,11.0,109.0,1202800.0,NaN,402600.0,1605400.0,2.6,0.838462,1.666667,51300
2,733,136122,PG0252,2017-01-08T10:05:00Z,2017-01-08T11:45:00Z,AAQ,SVO,Arrived,2017-01-08T10:08:00Z,2017-01-08T11:48:00Z,130,12,118,87.0,0.0,10.0,97.0,1065000.0,NaN,366000.0,1431000.0,2.6,0.746154,1.666667,51300
3,733,136130,PG0252,2017-02-13T10:05:00Z,2017-02-13T11:45:00Z,AAQ,SVO,Arrived,2017-02-13T10:07:00Z,2017-02-13T11:46:00Z,130,12,118,97.0,0.0,10.0,107.0,1190600.0,NaN,366000.0,1556600.0,2.6,0.823077,1.666667,51300
4,733,136131,PG0252,2017-01-28T10:05:00Z,2017-01-28T11:45:00Z,AAQ,SVO,Arrived,2017-01-28T10:07:00Z,2017-01-28T11:46:00Z,130,12,118,112.0,0.0,12.0,124.0,1373600.0,NaN,439200.0,1812800.0,2.6,0.953846,1.666667,51300
5,733,136132,PG0252,2017-02-15T10:05:00Z,2017-02-15T11:45:00Z,AAQ,SVO,Arrived,2017-02-15T10:09:00Z,2017-02-15T11:48:00Z,130,12,118,104.0,0.0,12.0,116.0,1274800.0,NaN,439200.0,1714000.0,2.6,0.892308,1.666667,51300
6,733,136135,PG0252,2017-02-01T10:05:00Z,2017-02-01T11:45:00Z,AAQ,SVO,Arrived,2017-02-01T10:08:00Z,2017-02-01T11:48:00Z,130,12,118,104.0,0.0,11.0,115.0,1276000.0,NaN,402600.0,1678600.0,2.6,0.884615,1.666667,51300
7,733,136146,PG0252,2017-02-09T10:05:00Z,2017-02-09T11:45:00Z,AAQ,SVO,Arrived,2017-02-09T10:08:00Z,2017-02-09T11:48:00Z,130,12,118,98.0,0.0,11.0,109.0,1201600.0,NaN,402600.0,1604200.0,2.6,0.838462,1.666667,51300
8,733,136159,PG0252,2017-01-04T10:05:00Z,2017-01-04T11:45:00Z,AAQ,SVO,Arrived,2017-01-04T10:07:00Z,2017-01-04T11:45:00Z,130,12,118,103.0,0.0,12.0,115.0,1261400.0,NaN,439200.0,1700600.0,2.6,0.884615,1.666667,51300
9,733,136164,PG0252,2017-01-07T10:05:00Z,2017-01-07T11:45:00Z,AAQ,SVO,Arrived,2017-01-07T10:07:00Z,2017-01-07T11:46:00Z,130,12,118,109.0,0.0,11.0,120.0,1335800.0,NaN,402600.0,1738400.0,2.6,0.923077,1.666667,51300


Посмотрим на уникальные рейсы, оставшиеся в нашем наборе данных, и количество их вылетов.

In [7]:
print(f'Количество уникальных рейсов: {df["flight_no"].nunique()} ед., в их числе: {df["flight_no"].unique()}')

Количество уникальных рейсов: 2 ед., в их числе: ['PG0252' 'PG0480']


Узнаем, сколько вылетов совершил каждый из представленных рейсов.

In [8]:
df.groupby('flight_no')['flight_id'].describe()

,count,mean,std,min,25%,50%,75%,max
flight_no,,,,,,,,
PG0252,59.0,136298.661017,117.208716,136119.0,136193.5,136284.0,136407.0,136486.0
PG0480,59.0,136768.220339,112.072801,136571.0,136663.5,136769.0,136859.0,136956.0


Видим, что оба рейса совершили одинаковое количество вылетов. С точки зрения количественной характеристики эффективности двух рейсов пока оба кажутся эффективными.

Посмотрим на другие показатели. Начнём с количества мест.

In [9]:
df.groupby('flight_no')['seats_count'].describe()

,count,mean,std,min,25%,50%,75%,max
flight_no,,,,,,,,
PG0252,59.0,130.0,0.0,130.0,130.0,130.0,130.0,130.0
PG0480,59.0,97.0,0.0,97.0,97.0,97.0,97.0,97.0


Рейс PG0252 осуществляет Boeing 737-300, суммарное количество мест в нём - 130 единиц, тогда как рейс PG0480 осуществляет Sukhoi Superjet-100 с суммарным количеством мест в 97 единиц. Потенциально, рейс PG0252 сможет перевозить больше пассажиров, чем рейс PG0480, однако стоит посмотреть на заполненность рейсов.

In [10]:
df.groupby('flight_no')[['fullness', 'ticket_total']].describe()

fullness                                                    \
             count      mean       std       min       25%       50%   
flight_no                                                              
PG0252        59.0  0.870143  0.068432  0.746154  0.823077  0.861538   
PG0480        59.0  0.929757  0.083522  0.659794  0.891753  0.979381   

                         ticket_total                                     \
                75%  max        count        mean       std   min    25%   
flight_no                                                                  
PG0252     0.923077  1.0         59.0  113.118644  8.896115  97.0  107.0   
PG0480     1.000000  1.0         59.0   90.186441  8.101670  64.0   86.5   

                                
             50%    75%    max  
flight_no                       
PG0252     112.0  120.0  130.0  
PG0480      95.0   97.0   97.0

Так, рейс PG0252 в среднем заполнен на 87%, тогда как рейс PG0480 заполнен на 93%. Разница между заполненностью рейсов не кажется критичной. Говоря об абсолютных значениях, в среднем рейс PG0480 перевозит около 113 пассажиров, в то время как рейс PG0480 перевозит около 90 пассажиров.

Потенциально важный момент в оценке прибыльности рейсов: их затраты на перелёт. Затраты будем оценивать как стоимость топлива на время пути рейса. Для этого понадобится время полёта умножить на расход топлива в час (в тоннах) и умножить на стоимость топлива за тонну. 

Посчитаем затраты для каждого рейса и проанализируем результаты.

In [11]:
df['flight_spend'] = df['flight_time_hours'] * df['fuel_consumption_per_hour'] * df['fuel_price_per_ton']
df.groupby('flight_no')['flight_spend'].describe()

,count,mean,std,min,25%,50%,75%,max
flight_no,,,,,,,,
PG0252,59.0,222300.0,2.935365e-11,222300.0,222300.0,222300.0,222300.0,222300.0
PG0480,59.0,76950.0,0.000000e+00,76950.0,76950.0,76950.0,76950.0,76950.0


Так, рейс PG0252 находится в пути 1,67 часов, при этом на час полёта он расходует 2,6 тонны топлива при стоимости 51300 рублей за тонну (информация о расходе топлива и стоимости топлива была взята из открытых источников). Так, один вылет рейса PG0252 оценивается в 222 300 рублей. 

Рейс PG0480 находится в пути 0.83 часа, при этом на час полёта он расходует 1.8 тонны топлива при стоимости 51300 рублей за тонну. Стоимость вылета этого рейса оценивается в 76 950 рублей. 

Очевидно, что рейс PG0480 несёт меньше расходов при совершении рейса.

Однако также стоит рассмотреть доходность рейсов.

Для оценки доходность будем считать показатель рентабельности затрат (ROS) как отношение дохода от рейса к затратам на него. Конечно, в данном проекте это будет лишь приближённой оценкой, так как данных о иных затратах в открытых источниках нет, поэтому предположим, что прочие затраты на рейсы равны.

Посчитаем доходность и посмотрим на результаты.

In [12]:
df['ros'] = df['total_amount'] / df['flight_spend']
df.groupby('flight_no')['ros'].describe()

,count,mean,std,min,25%,50%,75%,max
flight_no,,,,,,,,
PG0252,59.0,7.389007,0.593246,6.288799,6.941970,7.326136,7.820063,8.484031
PG0480,59.0,9.232431,0.855559,6.900585,8.672515,9.699805,9.945419,9.945419


Рейс PG0252 в среднем приносит 1 642 576.27 рублей, соответственно, рентабельность (суммарный доход / затраты) этого рейса составляет 7.37 ед.

Рейс PG0480 в среднем приносит 710 435.59 рублей, соответственно, рентабельность этого рейса составляет 9.26 ед.

Оба рейса доходны и окупают затраты, однако рентабельность рейса PG0480 выше рентабельности рейса PG0252 .

# Вывод.

При принятии решения о том, какой рейс лучше закрыть, при допущении, что прочие расходы на оба рейса сопоставимы, целесообразнее закрыть рейс PG0480: он хоть и немного рентабельнее своего коллеги, но в абсолютном выражении приносит больше прибыли. 

Если расходы на рейсы неравноценны (стоимость ТО, оплата труда и др.), то рентабельность стоит посчитать с учётом этих расходов и принять иное решение.


А если рейс в Новокузнецк летает без пассажиров, то отменить нужно его:)